[Source Code gốc lấy từ đây](https://www.kaggle.com/code/dinhduonghainam/benchmark-embeedings-model)

In [10]:
model_names =[
  'intfloat/multilingual-e5-base',
  "BAAI/bge-m3",
  'intfloat/multilingual-e5-large',
  'nampham1106/bkcare-embedding',
  "dangvantuan/vietnamese-embedding",
  "VoVanPhuc/sup-SimCSE-VietNamese-phobert-base",
  "keepitreal/vietnamese-sbert",
  "bkai-foundation-models/vietnamese-bi-encoder",
  "hiieu/halong_embedding",
  "vinai/phobert-base",
  "vinai/phobert-large",
  "vinai/phobert-base-v2",
]

datasets = ['STS-B', 'STS12', 'STS13', 'STS14', 'STS15', 'STS16', 'STS-Sickr']

In [11]:
import os
path_org = 'STS_output/'


In [12]:
import pandas as pd
# Adjust the approach to extract data directly from the columns of cosine_pearson and cosine_spearman
pearson_scores = {model: [] for model in model_names}
spearman_scores = {model: [] for model in model_names}

# Loop through each dataset and model, read the CSV and extract the needed coefficients directly
for dataset in datasets:
    dataset_path = os.path.join(path_org, dataset)
    for model in model_names:
        model_path = os.path.join(dataset_path, model.replace("/", "+"))
        csv_path = os.path.join(model_path, 'similarity_evaluation_sts-test_results.csv')
        if os.path.exists(csv_path):
            data = pd.read_csv(csv_path)
            # Extract cosine_pearson and cosine_spearman directly
            cosine_pearson = data['cosine_pearson'].values[0]
            cosine_spearman = data['cosine_spearman'].values[0]
        else:
            # Handle missing data case
            print("Not exist path", csv_path)
            continue

        # Append the scores to the respective lists in the dictionary
        pearson_scores[model].append(round(100*cosine_pearson,2))
        spearman_scores[model].append(round(100*cosine_spearman,2))

# Convert dictionaries to dataframes
df_pearson = pd.DataFrame(pearson_scores, index=datasets)
df_spearman = pd.DataFrame(spearman_scores, index=datasets)
df_pearson.loc['Mean'] = df_pearson.mean()
df_spearman.loc['Mean'] = df_spearman.mean()
# Transpose the dataframes to switch columns and rows
df_pearson_transposed = df_pearson.T
df_spearman_transposed = df_spearman.T

[dangvantuan/vietnamese-embedding](https://huggingface.co/dangvantuan/vietnamese-embedding) sử dụng bộ này để train nên loại trừ khỏi benchmark này

In [13]:
df_pearson_transposed = df_pearson_transposed[df_pearson_transposed.index != 'dangvantuan/vietnamese-embedding']
df_spearman_transposed = df_spearman_transposed[df_spearman_transposed.index != 'dangvantuan/vietnamese-embedding']

In [14]:
# Sort the transposed dataframes by the 'Mean' column in descending order
df_pearson_sorted = df_pearson_transposed.sort_values(by='Mean', ascending=False)
df_spearman_sorted = df_spearman_transposed.sort_values(by='Mean', ascending=False)


In [15]:
def bold_max(s):
    is_max = s == s.max()
    return ['font-weight: bold' if v else '' for v in is_max]

# Apply the function to each row of the dataframes
df_pearson_styled = df_pearson_sorted.style.apply(bold_max, axis=0)
df_spearman_styled = df_spearman_sorted.style.apply(bold_max, axis=0)

In [16]:
df_spearman_styled

,STS-B,STS12,STS13,STS14,STS15,STS16,STS-Sickr,Mean
VoVanPhuc/sup-SimCSE-VietNamese-phobert-base,81.430000,76.510000,79.190000,74.910000,81.720000,76.570000,76.450000,78.111429
keepitreal/vietnamese-sbert,80.160000,69.080000,80.990000,73.670000,82.810000,74.300000,73.400000,76.344286
nampham1106/bkcare-embedding,79.880000,72.660000,78.350000,70.740000,77.610000,75.140000,77.380000,75.965714
intfloat/multilingual-e5-large,80.690000,71.370000,69.960000,68.770000,81.440000,80.090000,75.120000,75.348571
BAAI/bge-m3,78.410000,70.720000,69.080000,68.650000,82.790000,79.280000,75.720000,74.950000
intfloat/multilingual-e5-base,76.760000,69.200000,67.260000,65.730000,79.350000,77.530000,72.400000,72.604286
hiieu/halong_embedding,74.540000,62.750000,71.410000,65.510000,78.660000,75.350000,70.860000,71.297143
bkai-foundation-models/vietnamese-bi-encoder,72.160000,63.860000,71.820000,66.200000,78.620000,74.240000,70.870000,71.110000
vinai/phobert-base-v2,53.420000,44.120000,52.120000,45.260000,57.920000,60.660000,58.800000,53.185714
vinai/phobert-base,50.650000,36.050000,46.350000,42.510000,55.400000,56.180000,59.190000,49.475714


In [17]:
df_pearson_styled

,STS-B,STS12,STS13,STS14,STS15,STS16,STS-Sickr,Mean
VoVanPhuc/sup-SimCSE-VietNamese-phobert-base,81.520000,85.020000,78.220000,75.940000,81.530000,75.390000,77.750000,79.338571
keepitreal/vietnamese-sbert,80.540000,78.580000,80.750000,76.980000,82.570000,73.210000,80.160000,78.970000
nampham1106/bkcare-embedding,80.630000,80.590000,78.060000,73.970000,77.580000,74.160000,81.230000,78.031429
intfloat/multilingual-e5-large,81.230000,77.890000,69.950000,69.150000,81.080000,79.510000,78.340000,76.735714
BAAI/bge-m3,78.140000,74.670000,69.100000,68.390000,82.430000,78.620000,78.300000,75.664286
intfloat/multilingual-e5-base,77.500000,74.670000,67.040000,66.350000,79.050000,77.070000,76.850000,74.075714
hiieu/halong_embedding,75.670000,67.080000,71.080000,68.290000,78.550000,75.040000,77.300000,73.287143
bkai-foundation-models/vietnamese-bi-encoder,73.300000,67.840000,71.690000,69.800000,78.400000,74.290000,76.010000,73.047143
vinai/phobert-base-v2,52.780000,40.020000,50.760000,46.640000,56.710000,57.860000,60.560000,52.190000
vinai/phobert-base,49.740000,29.670000,46.800000,43.690000,54.830000,53.670000,61.550000,48.564286
